In [15]:
# fetch arguments
# args <- commandArgs(trailingOnly = T)

library(data.table)

# args[1] <- '/data/r-workspace/strandseq_utils/H3JNHAFX3_MNIphotoconverted_200000_fixed.norm.txt.gz'
# args[2] <- '/data/r-workspace/strandseq_utils/GC_matrix_200000.txt'
# print(snakemake@params[["gc_matrix"]])
# open files
# counts <- data.table::fread(snakemake@input[["counts_scaled"]], header = T)
counts <- data.table::fread("/scratch/tweber/DATA/MC_DATA/2023-03-08-HCNGHAFX5/HGSVCpool2OPSfromFrozen2ul/counts/GC_correction/HGSVCpool2OPSfromFrozen2ul.txt.scaled.gz", header = T)
GC_matrix <- data.table::fread("../data/GC/GC_matrix_200000.txt", header = T)
print(GC_matrix)
# GC_matrix <- data.table::fread(args[2], header = T)
# save_path <- snakemake@output[["counts_scaled_gc"]]
# save_path <- args[3]

# check GC plots
# plot <- ifelse(is.na(args[4]), FALSE, TRUE)
# plot <- TRUE
# if (plot) {
#   # import libraries
#   library(ggplot2)
#   library(ggpubr)
# }


# check files
if (!all(c("cell", "chrom", "start", "end", "w", "c") %in% colnames(counts))) {
  message("count file does not contain required columns: 'cell', 'chrom', 'start', 'end', 'w', 'c'")
  message("Usage: Rscript GC_correction.R count-file.txt.gz gc-matrix.txt output.txt.gz")
  stop()
}
if (!all(c("chrom", "start", "end", "GC%") %in% colnames(GC_matrix))) {
  message("GC_matrix file does not contain required columns: 'chrom', 'start', 'end', 'GC%'")
  message("Usage: Rscript GC_correction.R count-file.txt.gz gc-matrix.txt output.txt.gz")
  stop()
}
if (!(all(unique(counts$chrom) %in% unique(GC_matrix$chrom)) &
  all(unique(counts$start) %in% unique(GC_matrix$start)) &
  all(unique(counts$end) %in% unique(GC_matrix$end)))) {
  message("bin features ('crhom', 'start', 'end') do not match between count file and GC matrix")
  message("make sure to choose files with identical bin sizes")
}


# green light message
# message(paste("\ncount file:", args[1]))
# message(paste("GC matrix file:", args[2]))
# message(paste("savepath:", args[3]))
message("preprocessing...\n")


#################
# Preprocessing #
#################
# min_reads <- snakemake@params[["gc_min_reads"]]
# n_subsample <- snakemake@params[["gc_n_subsample"]]
min_reads <- 5
n_subsample <- 1000

       Index chrom    start      end          PeakID 2 3 4 5     CpG%      GC%
    1:     1  chr1        0   200000     bin_200kb_1 0 0 0 0 0.012235 0.424517
    2:     2  chr1   200000   400000     bin_200kb_2 0 0 0 0 0.008464 0.405687
    3:     3  chr1   400000   600000     bin_200kb_3 0 0 0 0 0.008521 0.401892
    4:     4  chr1   600000   800000     bin_200kb_4 0 0 0 0 0.011152 0.420935
    5:     5  chr1   800000  1000000     bin_200kb_5 0 0 0 0 0.014692 0.448248
   ---                                                                        
15449: 15449  chrY 56400000 56600000 bin_200kb_15426 0 0 0 0     <NA>     <NA>
15450: 15450  chrY 56600000 56800000 bin_200kb_15427 0 0 0 0 0.028728 0.389592
15451: 15451  chrY 56800000 57000000 bin_200kb_15428 0 0 0 0 0.008219 0.406364
15452: 15452  chrY 57000000 57200000 bin_200kb_15429 0 0 0 0 0.008362 0.383905
15453: 15453  chrY 57200000 57227415            none 0 0 0 0     none     none


preprocessing...




In [16]:
counts

chrom,start,end,sample,cell,c,w,class,tot_count
<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>
chr1,0,200000,HGSVCpool2OPSfromFrozen2ul,HGSVCpool2OPSfromFrozen2uliTRU3C01,0,0,None,0
chr1,0,200000,HGSVCpool2OPSfromFrozen2ul,HGSVCpool2OPSfromFrozen2uliTRU3C02,0,0,WC,0
chr1,0,200000,HGSVCpool2OPSfromFrozen2ul,HGSVCpool2OPSfromFrozen2uliTRU3C03,0,0,CC,0
chr1,0,200000,HGSVCpool2OPSfromFrozen2ul,HGSVCpool2OPSfromFrozen2uliTRU3C04,0,0,WC,0
chr1,0,200000,HGSVCpool2OPSfromFrozen2ul,HGSVCpool2OPSfromFrozen2uliTRU3C05,0,0,WC,0
chr1,0,200000,HGSVCpool2OPSfromFrozen2ul,HGSVCpool2OPSfromFrozen2uliTRU3C06,0,0,WC,0
chr1,0,200000,HGSVCpool2OPSfromFrozen2ul,HGSVCpool2OPSfromFrozen2uliTRU3C07,0,0,WW,0
chr1,0,200000,HGSVCpool2OPSfromFrozen2ul,HGSVCpool2OPSfromFrozen2uliTRU3C08,0,0,WC,0
chr1,0,200000,HGSVCpool2OPSfromFrozen2ul,HGSVCpool2OPSfromFrozen2uliTRU3C09,0,0,WW,0


In [17]:
GC_matrix

Index,chrom,start,end,PeakID,2,3,4,5,CpG%,GC%
<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>
1,chr1,0,200000,bin_200kb_1,0,0,0,0,0.012235,0.424517
2,chr1,200000,400000,bin_200kb_2,0,0,0,0,0.008464,0.405687
3,chr1,400000,600000,bin_200kb_3,0,0,0,0,0.008521,0.401892
4,chr1,600000,800000,bin_200kb_4,0,0,0,0,0.011152,0.420935
5,chr1,800000,1000000,bin_200kb_5,0,0,0,0,0.014692,0.448248
6,chr1,1000000,1200000,bin_200kb_6,0,0,0,0,0.044136,0.627472
7,chr1,1200000,1400000,bin_200kb_7,0,0,0,0,0.045416,0.626861
8,chr1,1400000,1600000,bin_200kb_8,0,0,0,0,0.037295,0.574125
9,chr1,1600000,1800000,bin_200kb_9,0,0,0,0,0.040316,0.567934


In [5]:

counts <- merge(counts, GC_matrix[, c("chrom", "start", "GC%")], by = c("chrom", "start"), all.x = T)

# filter data for subsampling
c <- counts[counts$tot_count >= min_reads]
print(c)
c$`GC%` <- as.numeric(c$`GC%`)
c$log_count_norm <- log(c$tot_count) - log(median(c$tot_count))
not.na <- !is.na(c$`GC%`)
s <- c[not.na]

Empty data.table (0 rows and 10 cols): chrom,start,end,sample,cell,c...


In [10]:
s$GC_bin <- cut(s$`GC%`, breaks = c(quantile(s$`GC%`, probs = seq(0, 1, by = 1 / 10))), labels = seq(1, 10, by = 1), include.lowest = TRUE)


ERROR: Error in cut.default(s$`GC%`, breaks = c(quantile(s$`GC%`, probs = seq(0, : lengths of 'breaks' and 'labels' differ


In [14]:
c

chrom,start,end,sample,cell,c,w,class,tot_count,GC%,log_count_norm
<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<dbl>,<dbl>
